### Example Exploratory Notebook

Use this notebook to explore the data generated by the pipeline in your preferred programming language.

**Note**: This notebook is not executed as part of the pipeline.

In [0]:
import requests
import pandas as pd
from datetime import datetime

url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json",
    "Referer": "https://www.nseindia.com"
}

session = requests.Session()
session.get("https://www.nseindia.com", headers=headers)  # create cookies

r = session.get(url, headers=headers)

print("Status:", r.status_code)

data = r.json()
df = pd.DataFrame(data["data"])
df["date"] = datetime.today().strftime('%Y-%m-%d')
display(df)

In [0]:

spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.NSE_SET")
spark.sql("""
    CREATE TABLE IF NOT EXISTS workspace.NSE_SET.NIFTY50_DAILY (
        priority              BIGINT,
        symbol                STRING,
        identifier            STRING,
        open                  DOUBLE,
        dayHigh               DOUBLE,
        dayLow                DOUBLE,
        lastPrice             DOUBLE,
        previousClose         DOUBLE,
        change                DOUBLE,
        pChange               DOUBLE,
        ffmc                  DOUBLE,
        yearHigh              DOUBLE,
        yearLow               DOUBLE,
        totalTradedVolume     BIGINT,
        stockIndClosePrice    BIGINT,
        totalTradedValue      DOUBLE,
        lastUpdateTime        STRING,
        nearWKH               DOUBLE,
        nearWKL               DOUBLE,
        perChange365d         DOUBLE,
        perChange30d          DOUBLE,
        date365dAgo           STRING,
        date30dAgo            STRING,
        chartTodayPath        STRING,
        chart30dPath          STRING,
        chart365dPath         STRING,
        series                STRING,
        meta                  STRING,
        trade_date            date,
        load_date             date
    )
""")

In [0]:
spark_df = spark.createDataFrame(df)
from pyspark.sql.functions import current_date
load_date = current_date()
spark_df = spark_df.withColumnRenamed("date", "trade_date").withColumn("load_date", load_date)
from pyspark.sql.functions import col

# Delete rows with the same load_date before inserting new data
spark.sql(f"DELETE FROM workspace.NSE_SET.NIFTY50_DAILY WHERE load_date = '{spark_df.select('load_date').first()['load_date']}'")

spark_df.write.insertInto("workspace.NSE_SET.NIFTY50_DAILY", overwrite=False)

In [0]:
%sql
select * from workspace.NSE_SET.NIFTY50_DAILY